In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from numpy.random import seed
from pandas import read_csv, DataFrame
from sklearn.preprocessing import minmax_scale
from keras.optimizers import SGD
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Flatten,Dense,Dropout,Activation
from keras.layers.normalization import BatchNormalization
import numpy as np
import pandas as pd
import sys
import tensorflow as tf
from keras import *
import keras
print( keras.__version__)
from keras import backend as K
K.set_image_data_format('channels_last')
##K.set_image_dim_ordering('tf')
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
from keras.metrics import binary_accuracy
from keras.metrics import categorical_accuracy

keras.backend.backend()


In [7]:
img_width, img_height = 200, 200


train_data_dir      = '/content/data/TRAIN1.zip'
test_data_dir = '/content/data/TEST.zip'

batch_size = 2

In [8]:
# Step 1: Extract the datasets
def extract_zip(zip_path, extract_to):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

train_zip = "/content/data/TRAIN1.zip"
test_zip = "/content/data/TEST.zip"
train_dir = "/content/data/TRAIN"
test_dir = "/content/data/TEST"

extract_zip(train_zip, train_dir)
extract_zip(test_zip, test_dir)

In [9]:
# Step 2: Data preparation
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # Split training data for validation
)

In [10]:
test_datagen = ImageDataGenerator(rescale=1.0/255)

In [11]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset='training'
)


Found 584 images belonging to 1 classes.


In [12]:
validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

Found 145 images belonging to 1 classes.


In [13]:
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

Found 207 images belonging to 1 classes.


In [14]:
#  Build the model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D((2, 2)),
    Dropout(0.2),  # Reduce overfitting
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [15]:
#  Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),  # Adjust learning rate
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [17]:
#  Add callbacks
checkpoint_cb = ModelCheckpoint('best_model.keras', save_best_only=True)
early_stopping_cb = EarlyStopping(patience=10, restore_best_weights=True)

# Step 6: Train the model
history = model.fit(
    train_generator,
    epochs=50,
    validation_data=validation_generator,
    callbacks=[checkpoint_cb, early_stopping_cb]
)

Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


19/19 ━━━━━━━━━━━━━━━━━━━━ 44s 2s/step - accuracy: 0.8930 - loss: 2.1171 - val_accuracy: 1.0000 - val_loss: 1.1370
Epoch 2/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 81s 2s/step - accuracy: 1.0000 - loss: 0.9739 - val_accuracy: 1.0000 - val_loss: 0.5953
Epoch 3/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 39s 2s/step - accuracy: 1.0000 - loss: 0.5184 - val_accuracy: 1.0000 - val_loss: 0.3280
Epoch 4/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 46s 2s/step - accuracy: 1.0000 - loss: 0.2885 - val_accuracy: 1.0000 - val_loss: 0.1899
Epoch 5/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 42s 2s/step - accuracy: 1.0000 - loss: 0.1665 - val_accuracy: 1.0000 - val_loss: 0.1141
Epoch 6/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 79s 2s/step - accuracy: 1.0000 - loss: 0.1014 - val_accuracy: 1.0000 - val_loss: 0.0702
Epoch 7/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 43s 2s/step - accuracy: 1.0000 - loss: 0.0626 - val_accuracy: 1.0000 - val_loss: 0.0439
Epoch 8/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - accuracy: 1.0000 - loss: 0.0397 - val_accuracy: 1.0000 - val_loss: 0.0297
Epo

In [18]:
#  Evaluate the model
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {test_accuracy:.2f}")

7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 441ms/step - accuracy: 1.0000 - loss: 5.0720e-05
Test Accuracy: 1.00
